# KNN regression experiments

In class we learned about how KNN regression works, and tips for using KNN.  For example, we learned that data should be scaled when using KNN, and that extra, useless predictors should not be used with KNN.  Are these tips really correct?

In this notebook we run a bunch of tests to see how KNN is affect by the choice of k, distance function, scaling of the predictors, presence of useless predictors, and other things.

One experiment we do not run, and which would be interesting, is to see how KNN performance changes as a function of the size of the training set.

## INSTRUCTIONS

Enter code wherever you see # YOUR CODE HERE in code cells, or YOU TEXT HERE in markup cells.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt

In [ ]:
# set default figure size
plt.rcParams['figure.figsize'] = [8.0, 6.0]

In [ ]:
# code in this cell from: 
# https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to display/hide the code."></form>''')

### Read the data and take a first look at it

The diamonds dataset is good for testing KNN because it has many numeric features.  See https://www.kaggle.com/shivam2503/diamonds for information on the dataset.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/grbruns/cst383/master/diamonds.csv")    

In [ ]:
df.info()

Note that numeric features have different ranges.  For example, the median value of carat is 0.7, while the median value of depth is about 62.  Price has a much greater median value, but we will be using it as the target variable.

In [ ]:
df.describe()

### Prepare data for machine learning

We will use KNN regression to predict the price of a diamond from its physical features.

We use a subset of the data set for our training and test data.  Note that we keep an unscaled version of the data for one of the experiments we will run.

In [ ]:
# for repeatability
np.random.seed(0)   

In [ ]:
# select the predictor variables and target variables to be used with regression
predictors = ['carat', 'depth', 'table', 'x', 'y', 'z']
target = 'price'
X = df[predictors].values
y = df[target].values

In [ ]:
# KNN can be slow, so get a random sample of the full data set
indexes = np.random.choice(y.size, size=7000)
X_mini = X[indexes]
y_mini = y[indexes]

In [ ]:
# Split the data into training and test sets, and scale
scaler = StandardScaler()

# unscaled version (note that scaling is only used on predictor variables)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_mini, y_mini, test_size=0.30, random_state=42)

# scaled version
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [ ]:
# sanity check
print(X_train.shape)
print(X_train[:3])

## Baseline performance

_For regression problems, our baseline is the "blind" prediction that is just the average value of the target variable.  The blind prediction must be calculated using the training data.  Calculate and print the test set root mean squared error (test RMSE) using this blind prediction.
I have provided a function you can use for RMSE._

In [ ]:
def rmse(predicted, actual):
    return np.sqrt(((predicted - actual)**2).mean())

In [ ]:
# YOUR CODE HERE

## Performance with default hyperparameters

_Using the training set, train a KNN regression model using the ScikitLearn KNeighborsRegressor, and report on the test RMSE.  The test RMSE is the RMSE computed using the test data set._  

_When using the KNN algorithm, use algorithm='brute' to get the basic KNN algorithm._

In [ ]:
# YOUR CODE HERE

## Impact of K

_In class we discussed the relationship of the hyperparameter k to overfitting._

_I provided code to test KNN on k=1, k=3, k=5, ..., k=29.  For each value of k, compute the training RMSE and test RMSE.  The training RMSE is the RMSE computed using the training data.  Use the 'brute' algorithm, and Euclidean distance, which is the default.  You need to add the get_train_test_rmse() function._

In [ ]:
def get_train_test_rmse(regr, X_train, X_test, y_train, y_test):
    # YOUR CODE HERE

In [ ]:
n = 30
test_rmse = []
train_rmse = []
ks = np.arange(1, n+1, 2)
for k in ks:
    print(k, ' ', end='')
    regr = KNeighborsRegressor(n_neighbors=k, algorithm='brute')
    rmse_tr, rmse_te = get_train_test_rmse(regr, X_train, X_test, y_train, y_test)
    train_rmse.append(rmse_tr)
    test_rmse.append(rmse_te)
print('done')

In [ ]:
# sanity check
print('Test RMSE when k = 5: {:0.1f}'.format(test_rmse[2]))

_Using the training and test RMSE values you got for each value of k, find the k associated with the lowest test RMSE value. Print this k value and the associated lowest test RMSE value.  In other words, if you found that k=11 gave the lowest test RMSE, then print the value 11 and the test RMSE value obtained when k=11._

In [ ]:
def get_best(ks, rmse):
    # YOUR CODE HERE

best_k, best_rmse = get_best(ks, test_rmse)
print('best k = {}, best test RMSE: {:0.1f}'.format(best_k, best_rmse))

_Plot the test and training RMSE as a function of k, for all the k values you tried._

In [ ]:
# YOUR CODE HERE

### Comments

_In the markup cell below, write about what you learned from your plot.  I would expect two or three sentences, but what's most important is that you write something thoughtful._

YOUR TEXT HERE  

## Impact of distance metric

_Repeat what you did to test the impact of k, but this time use Manhattan distance as your distance metric.  Look at the options for KNeighborsRegressor() to see how to use Manhattan distance._

In [ ]:
# YOUR CODE HERE

_Print the value of k that gives the best test RMSE, and the test RMSE associated with that k, just as you did in the previous section._

In [ ]:
# YOUR CODE HERE

_Plot the training and test RMSE as a function of k, just as you did in the previous section.  Be sure to note that Manhattan distance was used in your plot title._

In [ ]:
# YOUR CODE HERE

### Comments

_Consider what you learned from your experiment, and write a little about it.  Think about how the results changed as a result of changing the distance function._

YOUR TEXT HERE

## Impact of noise predictors

_In class we heard that the KNN performance goes down if useless "noisy predictors" are present.  These are predictor that don't help in making predictions.  In this section, run KNN regression by adding one noise predictor to the data, then 2 noise predictors, then three, and then four.  For each, compute the training and test RMSE.  In every case, use k=10 as the k value and use Euclidean distance as the distance function._

_The add_noise_predictor() method makes it easy to add a predictor variable of random values to X_train or X_test._

In [ ]:
def add_noise_predictor(X):
    """ add a column of random values to 2D array X """
    noise = np.random.normal(size=(X.shape[0], 1))
    return np.hstack((X, noise))

_Hint: In each iteration of your loop, add a noisy predictor to both X_train and X_test.  You don't need to worry about rescaling the data, as the new noisy predictor is already scaled.  Don't modify X_train and X_test however, as you will be using them again._

In [ ]:
# YOUR CODE HERE

_Plot the percent increase in test RMSE as a function of the number of noise predictors.  The x axis will range from 0 to 4.  The y axis will show a percent increase in test RMSE._

_To compute percent increase in RMSE for n noise predictors, compute 100 * (rmse - base_rmse)/base_rmse, where base_rmse is the test RMSE with no noise predictors, and rmse is the test RMSE when n noise predictors have been added._

In [ ]:
# YOUR CODE HERE

### Comments

_Look at the results you obtained and add some thoughtful commentary._

YOUR TEXT HERE

## Impact of scaling

_In class we learned that we should scaled the training data before using KNN.  How important is scaling with KNN?  Repeat the experiments you ran before (like in the impact of distance metric section), but this time use unscaled data._

_Run KNN as before but use the unscaled version of the data.  You will vary k as before.  Use algorithm='brute' and Euclidean distance._

In [ ]:
# YOUR CODE HERE

_Print the best k and the test RMSE associated with the best k._

In [ ]:
# YOUR CODE HERE

_Plot training and test RMSE as a function of k.  Your plot title should note the use of unscaled data._

In [ ]:
# YOUR CODE HERE

### Comments

_Reflect on what happened and provide some short commentary, as in previous sections._

YOUR TEXT HERE

## Impact of algorithm

_We didn't discuss in class that there are variants of the KNN algorithm.  The main purpose of the variants is to be faster and to reduce that amount of training data that needs to be stored._

_Run experiments where you test each of the three KNN algorithms supported by Scikit-Learn: ball_tree, kd_tree, and brute.  In each case, use k=10 and use Euclidean distance._

In [ ]:
# YOUR CODE HERE

_Print the name of the best algorith, and the test RMSE achieved with the best algorithm._

In [ ]:
# YOUR CODE HERE

_Plot the test RMSE for each of the three algorithms as a bar plot._

In [ ]:
# YOUR CODE HERE

### Comments

_As usual, reflect on the results and add comments._

YOUR TEXT HERE

## Impact of weighting

_It was briefly mentioned in lecture that there is a variant of KNN in which training points are given more weight when they are closer to the point for which a prediction is to be made.  The 'weight' parameter of KNeighborsRegressor() has two possible values: 'uniform' and 'distance'.  Uniform is the basic algorithm._

_Run an experiment similar to the previous one.  Compute the test RMSE for uniform and distance weighting.  Using k = 10, the brute algorithm, and Euclidean distance._

In [ ]:
# YOUR CODE HERE

_Print the weighting the gave the lowest test RMSE, and the test RMSE it achieved._

In [ ]:
# YOUR CODE HERE

_Create a bar plot showing the test RMSE for the uniform and distance weighting options._

In [ ]:
# YOUR CODE HERE

### Comments

_As usual, reflect and comment._

YOUR TEXT HERE

## Conclusions

_Please provide at least a few sentences of commentary on the main things you've learned from the experiments you've run._

YOUR TEXT HERE